In [88]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchsampler import ImbalancedDatasetSampler
import matplotlib.pyplot as plt
import os
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import reduce
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import sklearn 
from torch.autograd import Variable
from sklearn.cluster import KMeans

import loaders as l
import models as m
import train_eval as te

import cv2
from sklearn.cluster import MeanShift, estimate_bandwidth
from PIL import Image
from matplotlib import cm
import pprint

from ImageClassification import helpers as h
import argparse
import time

In [89]:
saved_model_path='C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test_v2'
net = m.BaseNet_v2()
net.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [90]:
#Parameters 

cv2.destroyAllWindows() 
image = cv2.imread("C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test11.jpg")
winSize=(winW, winH) = (36, 36)
transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])
#face_detected=0
#layer=0
color_noFace=(255,0,0)
color_face=(0, 255, 0)
sleep=0.0015
sc=1.2
face_detected=0
layer=0
centers=[]
top_left=[]
all_layers=[]
all_prob=[]
all_x=[]
all_y=[]
nb_layers=0
sliding_step=10
toPrint=False#True if you want to see animations


In [91]:
cv2.destroyAllWindows()

In [92]:
cv2.destroyAllWindows()
#global_pyramid(image,scale,stepSize,windowSize,toPrint):
    
    
# loop over the image pyramid
for resized in h.pyramid(image, scale=sc):

    # loop over the sliding window for each layer of the pyramid

    for (x, y, window) in h.sliding_window(resized, stepSize=sliding_step, windowSize=(winW, winH)):
        # if the window does not meet our desired window size, ignore it
        if window.shape[0] != winH or window.shape[1] != winW:
            continue

        # window processing

        color=color_noFace
        sleep=0.005

        im = Image.fromarray(window.astype('uint8'), 'RGB')
        im=transform(im)
        im = im.unsqueeze(0)

        outputs=net(im)

        prob=torch.nn.functional.softmax(outputs, dim=1).detach()[0][1].numpy()

        y_pred_softmax = torch.log_softmax(outputs, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)

        prediction=np.array(y_pred_tags)

        if prediction==1 and prob>0.9:

            face_detected+=1
            #top left corner of the window
            x1=int(x*pow(sc,layer))
            y1=int(y*pow(sc,layer))
            top_left.append((x1,y1))

            #middle of the window
            x2=int(x1+(winW*pow(sc,layer)/2))
            y2=int(y1+(winH*pow(sc,layer)/2))
            centers.append((x2,y2))

            all_layers.append(layer)
            all_prob.append(prob)

            #print(new_face)
            color=color_face
            sleep=0.5


        #draw the window
        if toPrint : 
            clone = resized.copy()
            cv2.rectangle(clone, (x, y), (x + winW, y + winH), color, 2)
            cv2.imshow("Window", clone)
            cv2.waitKey(1)
            time.sleep(sleep)

    layer+=1
if toPrint :        
    print(face_detected)
    cv2.waitKey(1)
    cv2.destroyAllWindows() 

nb_layers=layer
#print(nb_layers)

In [93]:
#Shows a point for every sliding window classed as Face

cv2.destroyAllWindows() 

img_clone= image.copy()

for center in centers : 

    cv2.circle(img_clone, (int(center[0]), int(center[1])),2,color_face, 2)

cv2.imshow("Detected faces at step 1", img_clone)

cv2.waitKey(1)


-1

In [94]:
#Better analyze the zones previously detected by building a mini-pyramid around them
#It allows to get more points to analyze in the next cell and to remove some false positifs of the cell above

pp = pprint.PrettyPrinter(indent=2)
sliding_little_step = 2
top_left_precise = []
centers_precise = []
all_layers_precise = []
all_prob_precise = []

for i in range(len(centers)):
    
    p=top_left[i]
    l=all_layers[i]

    layer=0

    y_min=p[1]
    y_max=p[1]+int(winH*pow(sc,l))
    x_min=p[0]
    x_max=p[0]+int(winW*pow(sc,l))
    
    #mini pyramid centered around a possible face
    for (resized,x_crop,y_crop) in h.pyramid_v2(image[y_min:y_max,x_min:x_max], crop=5,layer=l,scale=sc):
    # loop over the sliding window for each layer of the pyramid
        for (x, y, window) in h.sliding_window(resized, stepSize=sliding_little_step, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            color=color_noFace
            sleep=0.005

            im = Image.fromarray(window.astype('uint8'), 'RGB')
            im=transform(im)
            im = im.unsqueeze(0)
            outputs=net(im)
            prob=torch.nn.functional.softmax(outputs, dim=1).detach()[0][1].numpy()
            #print(prob)
            y_pred_softmax = torch.log_softmax(outputs, dim = 1)
            _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
            prediction=np.array(y_pred_tags)
            new_face=[]

            if prediction==1 and prob>0.9:
                face_detected+=1
                #top left corner of the window
                x1 = int(x_min + x*pow(sc,layer)+x_crop)
                y1 = int(y_min + y*pow(sc,layer)+y_crop)
                top_left_precise.append((x1,y1))
                #middle of the window
                x2 = int(x1+(winW*pow(sc,layer)/2))
                y2 = int(y1+(winH*pow(sc,layer)/2))
                centers_precise.append((x2,y2))
                all_layers_precise.append(layer)
                all_prob_precise.append(prob)
                
                #print(new_face)
                color=color_face
                sleep=0.025
                
            #draw the window
            if toPrint :
                clone = resized.copy()
                cv2.rectangle(clone, (x, y), (x + winW, y + winH), color, 2)
                cv2.imshow("Window", clone)
                cv2.waitKey(1)
                time.sleep(sleep)
            
        layer+=1


In [97]:
#Clustering with MeanShift :it finds the number and position of of clusters and remove unclasteed points

cv2.destroyAllWindows() 
img_clone= image.copy()
data=centers_precise
   
bandwidth = estimate_bandwidth(data, quantile=0.2)

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True,cluster_all=False)
ms.fit(data)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)
#print(cluster_centers)

colors=[(255,0,0),(0,255,0),(0,0,255),(55,100,50),(25,130,50)]
i=0
for center in cluster_centers:
    
    cv2.circle(img_clone, (int(center[0]), int(center[1])),3,colors[i], 3)
    cv2.rectangle(img_clone, ( int(center[0])-36*int(nb_layers/10), int(center[1])-36*int(nb_layers/10) ) , (int(center[0])+36*int(nb_layers/10), int(center[1])+36*int(nb_layers/10) ), colors[i], 2)
    i+=1
    i%=len(colors)
    
for j in range(len(data)) : 
    cv2.circle(img_clone,data[j],1,(0,0,0), 1)
    
plt.show()
cv2.imshow("Layer", img_clone)
cv2.waitKey(1)


number of estimated clusters : 5


-1

In [96]:
#cluster filtering : we only show the cluster(s) with the biggest number(s) of elements

cv2.destroyAllWindows() 
img_clone= image.copy()


label_distribution=pd.Series(labels).value_counts()
print(label_distribution)
treshold=0.8*label_distribution.max()
faces_clusters=label_distribution[label_distribution>treshold].index.to_list()
face_centers=[cluster_centers[i] for i in faces_clusters]

for center in face_centers:
    
    #cv2.circle(img_clone, (int(center[0]), int(center[1])),2,color_face, 2)
    if nb_layers<8 : 
        nb_layers=8
    cv2.rectangle(img_clone, ( int(center[0])-36*int(nb_layers/10), int(center[1])-36*int(nb_layers/10) ) , (int(center[0])+36*int(nb_layers/10), int(center[1])+36*int(nb_layers/10) ),color_face, 2)


plt.show()
cv2.imshow("Faces", img_clone)
cv2.waitKey(1)

 0    880
-1    731
 1    317
 2    253
 3     14
dtype: int64


-1

In [ ]:
cv2.destroyAllWindows() 